title: My memberships (data discovery v1)        
author: Fabio Schmidt-Fischbach       
date: 2021-07-14      
region: EU       
link: https://docs.google.com/presentation/d/135m_6oLbYDpJAJfDQGnYB9uvy0NiUr4UErG48n0TpZo/edit?usp=sharing        
tags: ux, design, app navigation, premium, memberships, benefits, myaccount, explore   
summary: "This slide deck investigates basic app usage patterns (ios, android) around explore, membership benefits and myaccount. 19% of weekly logins visit explore at least once per week. This ranks lowest compared to myaccount (23%), cards (26%) or spaces (30%). Like most app activity, visits peak for young users and then stabilize over time. Benefits list is an important and frequently used feature in the app (5-10% of weekly logins visit the screen each week at least once). It is especially popular during onboarding peeking at almost 30pp of weekly logins visiting the screen at least once in the first week after KYC.  Benefits are usually found via explore (70% of all visits). 15% are coming via myaccount, the rest is not clear. Users engage quite regularly with content on benefits screen (up to a third of users do)."

In [1]:
import pandas as pd
import altair as alt

## General usage

In [ ]:
query = """



select date_trunc('week', collector_date) as week, 
			case when zup.market in ('DEU','ESP','ITA','FRA','AUT') then zup.market else 'RoE' end as market, 
			zup.product_id, 
			age_group, 
			gender, 
			case when se_property = 'benefits' then 'Account benefits'
			when se_property in ('credit_create','overdraft_manage',
									'overdraft_create','fixedterm_create',
									'credit_manage','credit_redirection_fra',
									'fixedterm_manage','instalment_manage','instalment_create')	then 'Bank products'
		    when se_property = 'referral' then 'Friend referral'
		    when se_property in ('N26 Smart', 'N26 Metal', 'N26 You','N26 Business You','N26 Business Metal','N26 Business Smart') then 'Upgrade'
		    when se_property = 'additional_cards' then 'Additional card'
		    when se_property in ('WLsr40PRs459at5OF8MOY','j58L1jWIbizZvChxIxOYt'
		    					,'2gIwTwcKuLKdyei3BkO5ku','5xlWkymoOV7bA8SPkuHSSc'
		    					,'7idiTHtfh4T0IviI6GoJ0K','1M4vswiRCOK5r76ne9JPHE') then 'Highlight'
		    when se_property like 'explore_benefits%' then 'Explore tab benefits'
		    end as type,
		    datediff('month', kyc_first_completed::date, collector_date::date) as age, 
			count(distinct dbt.snowplow.user_created) as users, 
			count(1) as visits
from dbt.snowplow 
inner join dbt.zrh_user_product as zup 	
	on zup.user_created = dbt.snowplow.user_created 
	and collector_date between subscription_valid_from and subscription_valid_until 
inner join dbt.zrh_users as zu on zu.user_created = zup.user_created 
where se_action='explore' and se_label = 'click'
	and collector_date >= '2021-01-01'
group by 1,2,3,4,5,6,7

"""

"""
select date_trunc('week', collector_date) as week, 
			case when zup.market in ('DEU','ESP','ITA','FRA','AUT') then zup.market else 'RoE' end as market, 
			zup.product_id, 
			age_group, 
			gender, 
			'view' as type,
		    datediff('month', kyc_first_completed::date, collector_date::date) as age, 
			count(distinct dbt.snowplow.user_created) as users, 
			count(1) as visits
from dbt.snowplow 
inner join dbt.zrh_user_product as zup 	
	on zup.user_created = dbt.snowplow.user_created 
	and collector_date between subscription_valid_from and subscription_valid_until 
inner join dbt.zrh_users as zu on zu.user_created = zup.user_created 
where se_action='explore' and se_label = 'view'
	and collector_date >= '2021-01-01'
group by 1,2,3,4,5,6,7

"""

"""
select date_trunc('week',collector_tstamp) as week, 
		case when gp.country_tnc_legal in ('DEU','ESP','ITA','FRA','AUT') then gp.country_tnc_legal else 'RoE' end as market, 
		gp.product_id,
		age_group, 
		gender, 
		'Partnership' as type, 
		datediff('month', kyc_first_completed::date, collector_tstamp::date) as age, 
		count(distinct gp.user_created) as users,
		count(1) as visits 
from dbt.gp_explore_tab_events as gp
inner join dbt.zrh_users using (user_created) 
where 1=1 and collector_tstamp >= '2021-01-01'
group by 1,2,3,4,5,6,7


"""


"""

with x as ( 

select date_trunc('week', collector_date) as week, 
			case when zup.market in ('DEU','ESP','ITA','FRA','AUT') then zup.market else 'RoE' end as market, 
			zup.product_id, 
			age_group, 
			gender, 
		    datediff('month', kyc_first_completed::date, collector_date::date) as age, 
			dbt.snowplow.user_created
from dbt.snowplow 
inner join dbt.zrh_user_product as zup 	
	on zup.user_created = dbt.snowplow.user_created 
	and collector_date between subscription_valid_from and subscription_valid_until 
inner join dbt.zrh_users as zu on zu.user_created = zup.user_created 
where se_action='explore' and se_label = 'click'
	and collector_date >= '2021-01-01'

union all 

select  date_trunc('week',collector_tstamp) as week, 
		case when gp.country_tnc_legal in ('DEU','ESP','ITA','FRA','AUT') then gp.country_tnc_legal else 'RoE' end as market, 
		gp.product_id,
		age_group, 
		gender, 
		datediff('month', kyc_first_completed::date, collector_tstamp::date) as age, 
		gp.user_created
from dbt.gp_explore_tab_events as gp
inner join dbt.zrh_users using (user_created) 
where 1=1 and collector_tstamp >= '2021-01-01'

)


select week, market, product_id, age_group, gender, age, count(distinct user_created) as users 
from x 
group by 1,2,3,4,5,6


"""

In [21]:
df = pd.read_csv("clicks_main.csv")
d = pd.read_csv("partnerships.csv")

df = df.append(d).loc[df["type"].isna() == False, :]

views = pd.read_csv("views.csv")
views = views.rename(columns={"users": "view_users", "visits": "view_clicks"})

views = views.drop(columns=["type"])

df = df.merge(
    views,
    on=["week", "market", "product_id", "age_group", "gender", "age"],
    how="right",
)

df = df.groupby(["week", "type"]).agg("sum").reset_index()

df["perc"] = 100 * df["users"] / df["view_users"]

alt.Chart(df).mark_line().encode(
    x=alt.X("week:T", axis=alt.Axis(title="Calendar week")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of explore visitors")),
    color="type:N",
).properties(width=700, height=500, title="What are people doing on explore?")

alt.Chart(...)

In [3]:
df = pd.read_csv("clicks_main.csv")
d = pd.read_csv("partnerships.csv")

df = df.append(d).loc[df["type"].isna() == False, :]

views = pd.read_csv("views.csv")
views = views.rename(columns={"users": "view_users", "visits": "view_clicks"})

views = views.drop(columns=["type"])

df = df.merge(
    views,
    on=["week", "market", "product_id", "age_group", "gender", "age"],
    how="right",
)

d = {
    "BUSINESS_SMART": "Smart",
    "BUSINESS_BLACK": "You",
    "BLACK_CARD_MONTHLY": "You",
    "BUSINESS_METAL": "Metal",
    "METAL_CARD_MONTHLY": "Metal",
    "FLEX_ACCOUNT_MONTHLY": "Flex",
    "STANDARD": "Standard",
    "BUSINESS_CARD": "Standard",
    "SMART": "Smart",
}

df["product_id"] = df["product_id"].map(d)

df = df.groupby(["type", "product_id", "week"]).agg("sum").reset_index()

df = df.groupby(["type", "product_id"]).agg("mean").reset_index()

df["perc"] = round(100 * df["users"] / df["view_users"])


# Configure common options
base = (
    alt.Chart(df)
    .encode(
        alt.X("product_id:O", scale=alt.Scale(paddingInner=0)),
        alt.Y("type:O", scale=alt.Scale(paddingInner=0)),
    )
    .properties(width=500, height=500)
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color(
        "perc:Q",
        scale=alt.Scale(scheme="viridis"),
        legend=alt.Legend(direction="horizontal"),
    )
)

# Configure text
text = base.mark_text(baseline="middle").encode(
    text="perc:Q",
)

# Draw the chart
heatmap + text

alt.LayerChart(...)

In [36]:
df = pd.read_csv("union.csv")

views = pd.read_csv("views.csv")
views = views.rename(columns={"users": "view_users", "visits": "view_clicks"})

views = views.drop(columns=["type"])

views = views.loc[views["week"] != "2021-06-14 00:00:00", :]

df = df.merge(
    views,
    on=["week", "market", "product_id", "age_group", "gender", "age"],
    how="right",
)

d = {
    "BUSINESS_SMART": "Smart",
    "BUSINESS_BLACK": "You",
    "BLACK_CARD_MONTHLY": "You",
    "BUSINESS_METAL": "Metal",
    "METAL_CARD_MONTHLY": "Metal",
    "FLEX_ACCOUNT_MONTHLY": "Flex",
    "STANDARD": "Standard",
    "BUSINESS_CARD": "Standard",
    "SMART": "Smart",
}

df["product_id"] = df["product_id"].map(d)

df = df.groupby(["week", "product_id"]).agg("sum").reset_index()

df["perc"] = 100 * df["users"] / df["view_users"]

alt.Chart(df).mark_line().encode(
    x=alt.X("week:T", axis=alt.Axis(title="Calendar week")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of explore visitors that click")),
    color="product_id:N",
).properties(
    width=400, height=500, title="% of users on explore that click on an element"
)

alt.Chart(...)